<a href="https://colab.research.google.com/github/adhish-bahl/2347203_NNDL_Lab/blob/main/2347203_Lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Importing Libraries**

In [53]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

#### **Loading Dataset**

In [54]:
(ds_train, ds_test), ds_info = tfds.load('kmnist', split=['train', 'test'], shuffle_files=True, as_supervised=True, with_info=True)

#### **Data Preprocessing**

**Normalize the data (pixel values between 0 and 1)**

In [55]:
def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255.0, label

In [56]:
batch_size = 32
ds_train = ds_train.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_train = ds_train.shuffle(1000).batch(batch_size)

ds_test = ds_test.map(normalize_img, num_parallel_calls=tf.data.experimental.AUTOTUNE)
ds_test = ds_test.batch(batch_size)

**Convert dataset to numpy arrays**

In [57]:
def dataset_to_numpy(dataset):
    X, y = [], []
    for images, labels in dataset:
        X.extend(images.numpy())
        y.extend(labels.numpy())
    return np.array(X), np.array(y)

**Convert training and test datasets to NumPy arrays**

In [58]:
X_train, y_train = dataset_to_numpy(ds_train)
X_test, y_test = dataset_to_numpy(ds_test)

**Split the training data into training and validation sets (80% training, 20% validation)**

In [59]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

**Flatten the images to shape (batch_size, 784)**

In [60]:
X_train = X_train.reshape(-1, 784)
X_val = X_val.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

**Determine the centers using K-means clustering**

In [61]:
num_units = 100
kmeans = KMeans(n_clusters=num_units, random_state=42).fit(X_train)
centers = kmeans.cluster_centers_

#### **Radial Basis Function (RBF) Network**

In [62]:
class RBFNetwork(tf.keras.Model):
    def __init__(self, num_units, num_classes, centers):
        super(RBFNetwork, self).__init__()
        self.num_units = num_units
        self.num_classes = num_classes
        self.centers = tf.Variable(centers, trainable=False)
        self.beta = tf.Variable(tf.ones([self.num_units]), trainable=True)
        self.W = tf.Variable(tf.random.normal([self.num_units, self.num_classes]))

    def rbf(self, X):
        diff = tf.expand_dims(X, 1) - tf.expand_dims(self.centers, 0)
        distance_squared = tf.reduce_sum(tf.square(diff), axis=-1)
        return tf.exp(-self.beta * distance_squared)

    def call(self, X):
        rbf_output = self.rbf(X)
        return tf.nn.softmax(tf.matmul(rbf_output, self.W))

num_classes = 10
rbf_network = RBFNetwork(num_units, num_classes, centers)

learning_rate = 0.01
rbf_network.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                    metrics=['accuracy'])

#### **Training**

In [ ]:
history = rbf_network.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100)

Epoch 1/100
   1/1500 ━━━━━━━━━━━━━━━━━━━━ 6:33 262ms/step - accuracy: 0.0938 - loss: 2.3026

/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.1022 - loss: 2.3026 - val_accuracy: 0.1023 - val_loss: 2.3026
Epoch 2/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.1006 - loss: 2.3026 - val_accuracy: 0.1023 - val_loss: 2.3026
Epoch 3/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - accuracy: 0.0990 - loss: 2.3026 - val_accuracy: 0.1023 - val_loss: 2.3026
Epoch 4/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.1045 - loss: 2.3026 - val_accuracy: 0.1023 - val_loss: 2.3026
Epoch 5/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.1014 - loss: 2.3026 - val_accuracy: 0.1023 - val_loss: 2.3026
Epoch 6/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - accuracy: 0.1022 - loss: 2.3026 - val_accuracy: 0.1023 - val_loss: 2.3026
Epoch 7/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 6ms/step - accuracy: 0.1021 - loss: 2.3026 - val_accuracy: 0.1023 - val_loss: 2.3026
Epoch 8/100
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 11s 7ms/step - accuracy: 0.1014 - loss: 2.302

#### **Evaluation**

**Compute accuracy**

In [ ]:
test_loss, test_accuracy = rbf_network.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

**Print final training accuracy**

In [ ]:
final_train_accuracy = history.history['accuracy'][-1]
print("Final Training Accuracy:", final_train_accuracy)

**Plot training and validation accuracy**

In [ ]:
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.show()

**Plot training and validation loss**

In [ ]:
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.show()

**Plot confusion matrix**

In [ ]:
y_pred = np.argmax(rbf_network.predict(X_test), axis=1)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

plt.figure(figsize=(10, 7))
ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.arange(num_classes)).plot(cmap='Blues')
plt.title('Confusion Matrix')
plt.show()
